In [1]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import os
import pickle
import time

In [1]:
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import load_model
import cv2
import pickle
import time
import csv
from datetime import datetime
import mysql.connector

# connect to your MySQL database
mydb = mysql.connector.connect(
  host="localhost",  # replace with your host
  user="root",  # replace with your username
  password="MyNewPass",  # replace with your password
  database="attendance"  # replace with your database name
)

# Create the CSV file and write the headers
with open('attendance.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Name", "Date", "Time"])

# Load label to categorical
with open('label_to_categorical', 'rb') as label:
    label_to_categorical = pickle.load(label)

# Load the VGG19 model
model = load_model('Best.h5')

# Load the Haar cascade xml file for face detection
front_face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Video capture
capture = cv2.VideoCapture(0)
start_time = time.time()
frame_count = 0
IMAGE_SIZE = 224

while True:
    success, frame = capture.read()
    if not success:
        break

    faces = front_face_cascade.detectMultiScale(frame, 1.3, 5)
    frame_count += 1

    if type(faces) is np.ndarray:
        for (x, y, w, h) in faces:
            face_crop = frame[y:y+h, x:x+w]
            face_crop = cv2.resize(face_crop, (IMAGE_SIZE, IMAGE_SIZE))
            face_crop = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
            face_crop = np.expand_dims(face_crop, axis=0)
            face_crop = preprocess_input(face_crop)

            scores = model.predict(face_crop)
            class_id = np.argmax(scores)
            name = 'Unknown Person'

            if scores[0][class_id] > 0.99:
                name = label_to_categorical[0][class_id]
                text = "{}: {:.2f}".format(name, scores[0][class_id])
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                # Save the attendance data
                now = datetime.now()
                date_string = now.strftime("%Y-%m-%d")
                time_string = now.strftime("%H:%M:%S")
                with open('attendance.csv', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([name, date_string, time_string])

                # Insert the attendance data into the MySQL database
                mycursor = mydb.cursor()
                sql = "INSERT INTO attendance (name, date, time) VALUES (%s, %s, %s)"
                val = (name, date_string, time_string)
                mycursor.execute(sql, val)
                mydb.commit()

            else:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(frame, name, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    elapsed_time = time.time() - start_time
    fps = frame_count / elapsed_time
    cv2.putText(frame, 'FPS : ' + str(round(fps, 2)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    cv2.imshow('Face Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'mysql'

In [2]:
names = os.listdir('newdata/')
labels = [i for i in range (len(names))]
label_deict = dict(zip(names, labels))

In [5]:
with open ('label_to_categorical','rb') as label:
    label_to_categorical = pickle.load(label)
    print(label_to_categorical[0])

0                    Mostafa
1    pins_Alexandra Daddario
2            pins_Bill Gates
3     pins_Cristiano Ronaldo
4          pins_Lionel Messi
Name: 0, dtype: object


In [7]:
model = load_model('Best.h5')
front_face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
profil_face_cascade = cv2.CascadeClassifier('haarcascade_profileface.xml')

In [12]:
def face_extractor(img):
    # function detects faces and return the cropped face
    # If no face detected, it returns the input image
    # gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    faces=face_cascade.detectMultiScale(img,1.3,5)

    if faces == ():
        
        return None

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face=img[y:y+h, x:x+w]

    return cropped_face

In [6]:
capture = cv2.VideoCapture(0)
count = 0
start_time = time.time()
frame_count = 0
IMAGE_SIZE = 100
while True:
        
    success, frame = capture.read()
    
    if not success:
        break
        
    front_faces = front_face_cascade.detectMultiScale(frame, 1.3, 5)
    profil_faces = profil_face_cascade.detectMultiScale(frame,1.3,5)
    frame_count += 1 
    
    for (x,y,w,h) in front_faces:
        
        if type(front_faces) is np.ndarray:
            
            set_  = frame[y: y+h, x: x+w]
            resize = cv2.resize(set_,(IMAGE_SIZE,IMAGE_SIZE))
            gray = cv2.cvtColor(resize, cv2.COLOR_BGR2RGB)
            img_array=np.array(gray)
            scaled = gray / 255.0
            img_array=np.expand_dims(scaled,axis=0)
            scores = model.predict(img_array)
            target = np.argmax(scores)
      
            name = 'Unknown Preson'
            
            if (scores[0][0] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][0], scores[0][0])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
            elif (scores[0][1] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][1], scores[0][1])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
            elif (scores[0][2] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][2], scores[0][2])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
            elif (scores[0][3] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][3], scores[0][3])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
            elif (scores[0][4] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][4], scores[0][4])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
                
            elif (scores[0][5] > 0.1):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                cv2.putText(frame, name, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  
            else:
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                cv2.putText(frame,name,(x, y-5),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
                
    for (x,y,w,h) in profil_faces:
        
        if type(profil_faces) is np.ndarray:
            
            set_  = frame[y: y+h, x: x+w]
            resize = cv2.resize(set_,(IMAGE_SIZE,IMAGE_SIZE))
            gray = cv2.cvtColor(resize, cv2.COLOR_BGR2RGB)
            img_array=np.array(gray)
            scaled = gray / 255.0
            img_array=np.expand_dims(scaled,axis=0)
            scores = model.predict(img_array)
            target = np.argmax(scores)
      
            name = 'Unknown Preson'
            
            if (scores[0][0] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][0], scores[0][0])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
            elif (scores[0][1] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][1], scores[0][1])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
            elif (scores[0][2] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][2], scores[0][2])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
            elif (scores[0][3] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][3], scores[0][3])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
            elif (scores[0][4] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][4], scores[0][4])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
                
            elif (scores[0][5] > 0.1):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                cv2.putText(frame, name, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  
            else:
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                cv2.putText(frame,name,(x, y-5),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

    

    elapsed_time = time.time() - start_time
    fps = frame_count / elapsed_time
    cv2.putText(img=frame, text='FPS : ' + str(round(fps, 2)), org=(10, 20), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255,255,255), thickness=1)

    # Show the frame
    cv2.imshow('Face Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 60ms/step


In [8]:
capture = cv2.VideoCapture(0)
count = 0
start_time = time.time()
frame_count = 0
IMAGE_SIZE = 100
while True:
        
    success, frame = capture.read()
    
    if not success:
        break
        
    faces = front_face_cascade.detectMultiScale(frame, 1.3, 5)
    frame_count += 1 
    
    if type(faces) is np.ndarray:
        
        for (x,y,w,h) in faces:

            set_  = frame[y: y+h, x : x+w]
            resize = cv2.resize(set_,(IMAGE_SIZE,IMAGE_SIZE))
            gray = cv2.cvtColor(resize, cv2.COLOR_BGR2RGB)
            img_array=np.array(gray)
            scaled = gray / 255.0
            img_array=np.expand_dims(scaled,axis=0)
            scores = model.predict(img_array)
            target = np.argmax(scores)
            print(scores)
            print(target)
            name = 'Unknown Person'


            if (scores[0][0] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][0], scores[0][0])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
            elif (scores[0][1] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][1], scores[0][1])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
            elif (scores[0][2] >= 0.9):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                text = "{}: {:.2f}".format(label_to_categorical[0][2], scores[0][2])
                cv2.putText(frame, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  
                
            elif (scores[0][3] > 0.1):
                
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
                cv2.putText(frame, name, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  

            else:
                cv2.rectangle(img=frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 255), thickness=2)
                cv2.putText(frame,name,(x,y),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,255),2)

    

    elapsed_time = time.time() - start_time
    fps = frame_count / elapsed_time
    cv2.putText(img=frame, text='FPS : ' + str(round(fps, 2)), org=(10, 20), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255,255,255), thickness=1)

    # Show the frame
    cv2.imshow('Face Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 214ms/step
[[2.0458553e-07 9.3451426e-06 7.6399743e-07 4.1726504e-02 9.5826316e-01]]
4
1/1 [==============================] - 0s 59ms/step
[[2.3120765e-06 9.2771458e-08 2.6799819e-06 9.1373402e-01 8.6260892e-02]]
3
1/1 [==============================] - 0s 72ms/step
[[2.1888595e-06 7.5690215e-05 2.4768484e-05 5.7327408e-01 4.2662326e-01]]
3
1/1 [==============================] - 0s 54ms/step
[[1.8346956e-06 9.0876220e-08 2.0899738e-06 9.3971461e-01 6.0281325e-02]]
3
1/1 [==============================] - 0s 67ms/step
[[1.6478147e-06 9.2169518e-07 1.4557844e-06 9.2098361e-01 7.9012454e-02]]
3
1/1 [==============================] - 0s 54ms/step
[[2.9405132e-06 5.6696877e-05 4.1824562e-05 2.9918289e-01 7.0071560e-01]]
4
1/1 [==============================] - 0s 55ms/step
[[1.6598523e-06 1.3503566e-06 2.5364500e-06 7.2951752e-01 2.7047694e-01]]
3
1/1 [==============================] - 0s 55ms/step
[[3.3368533e-06 8.8832305e-05 7.6851802e-06 5.04628